In [4]:
import pandas as pd
import matplotlib.pyplot as plt
from reportlab.lib.pagesizes import letter
from reportlab.lib import colors
from reportlab.lib.units import inch
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Paragraph, Spacer, Image
from reportlab.lib.styles import getSampleStyleSheet
import os

# Load data
df = pd.read_excel("matched_municipal_sites.xlsx", sheet_name="Individual Sites")

In [18]:
import pandas as pd
import matplotlib.pyplot as plt
from reportlab.lib.pagesizes import letter
from reportlab.lib import colors
from reportlab.lib.units import inch
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Paragraph, Spacer, Image
from reportlab.lib.styles import getSampleStyleSheet
import os

def generate_town_reports(data, town_names, output_dir, maps_dir):
    # Ensure the output directory exists
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Define styles for ReportLab
    styles = getSampleStyleSheet()
    title_style = styles["Title"]
    title_style.fontSize = 16
    title_style.leading = 20

    # Process each town individually
    for town_name in town_names:
        # Create a folder for the current town
        town_folder = os.path.join(output_dir, town_name)
        if not os.path.exists(town_folder):
            os.makedirs(town_folder)
        
        # Filter data for the current town
        town_data = data[data['GEOCITY'] == town_name]
        
        # Generate the summary table data
        summary_table = town_data.groupby('Matched Place Name').agg(
            count=('Matched Place Name', 'size'),
            maxArrayPanelsCount=('solarPotential.maxArrayPanelsCount', 'sum'),
            nominalPowerW=('Nominal Power (W)', 'sum'),
            predictedMeanAnnualPower=('Predicted Mean Annual Power (kWH/year)', 'sum'),
            housesPowered=('Houses Powered', 'sum'),
            annualCO2Savings=('Annual CO2 Generation savings (ton)', 'sum')
        ).reset_index()

        # Sort by "Houses Powered" in descending order
        summary_table = summary_table.sort_values(by="housesPowered", ascending=False)

        # Round numerical values and remove decimals
        summary_table["maxArrayPanelsCount"] = summary_table["maxArrayPanelsCount"].round(0).astype(int)
        summary_table["nominalPowerW"] = summary_table["nominalPowerW"].round(0).astype(int)
        summary_table["predictedMeanAnnualPower"] = summary_table["predictedMeanAnnualPower"].round(0).astype(int)
        summary_table["housesPowered"] = summary_table["housesPowered"].round(0).astype(int)
        summary_table["annualCO2Savings"] = summary_table["annualCO2Savings"].round(0).astype(int)
        
        # Format numbers with commas
        summary_table["maxArrayPanelsCount"] = summary_table["maxArrayPanelsCount"].apply(lambda x: f"{x:,}")
        summary_table["nominalPowerW"] = summary_table["nominalPowerW"].apply(lambda x: f"{x:,}")
        summary_table["predictedMeanAnnualPower"] = summary_table["predictedMeanAnnualPower"].apply(lambda x: f"{x:,}")
        summary_table["housesPowered"] = summary_table["housesPowered"].apply(lambda x: f"{x:,}")
        summary_table["annualCO2Savings"] = summary_table["annualCO2Savings"].apply(lambda x: f"{x:,}")
        
        # Create charts for the town
        charts = []
        
        # Chart 1: Top 10 Sites by Houses Powered
        top_houses_powered = town_data.groupby('Matched Place Name')['Houses Powered'].sum().nlargest(10)
        plt.figure(figsize=(10, 6))
        top_houses_powered.plot(kind='barh', color='skyblue')
        plt.title(f"{town_name} - Top 10 Sites by Houses Powered")
        plt.xlabel("Houses Powered")
        plt.gca().invert_yaxis()
        plt.tight_layout()  # Adjust layout to prevent label cutoff
        chart_path = os.path.join(town_folder, f"{town_name}_houses_powered_chart.png")
        plt.savefig(chart_path)
        charts.append(chart_path)
        plt.close()

        # Chart 2: Top 10 Sites by Nominal Power (W)
        top_nominal_power = town_data.groupby('Matched Place Name')['Nominal Power (W)'].sum().nlargest(10)
        plt.figure(figsize=(10, 6))
        top_nominal_power.plot(kind='barh', color='skyblue')
        plt.title(f"{town_name} - Top 10 Sites by Nominal Power (W)")
        plt.xlabel("Nominal Power (W)")
        plt.gca().invert_yaxis()
        plt.tight_layout()  # Adjust layout to prevent label cutoff
        chart_path = os.path.join(town_folder, f"{town_name}_nominal_power_chart.png")
        plt.savefig(chart_path)
        charts.append(chart_path)
        plt.close()

        # Chart 3: Distribution of Site Grouping
        top_grouping = town_data['Matched Place Name'].value_counts().nlargest(10)
        plt.figure(figsize=(10, 6))
        top_grouping.plot(kind='barh', color='skyblue')
        plt.title(f"{town_name} - Top 10 Grouped Sites")
        plt.xlabel("Number of Arrays")
        plt.gca().invert_yaxis()
        plt.tight_layout()  # Adjust layout to prevent label cutoff
        chart_path = os.path.join(town_folder, f"{town_name}_site_grouping_chart.png")
        charts.append(chart_path)
        plt.savefig(chart_path)
        plt.close()

        # Create PDF report for the town
        pdf_file_path = os.path.join(town_folder, f"{town_name}_report.pdf")
        doc = SimpleDocTemplate(pdf_file_path, pagesize=letter)

        # Map image path
        map_image_path = os.path.join(maps_dir, f"{town_name}_map.png")
        if not os.path.exists(map_image_path):
            print(f"Map for {town_name} not found at {map_image_path}. Skipping map image.")
            map_image_path = None

        # Prepare data for ReportLab table
        table_data = [
            ["Matched Place Name", "Count", "Max Array Panels\nCount", 
             "Nominal Power\n(W)", "Predicted Mean\nAnnual Power\n(kWh/year)", 
             "Houses\nPowered", "Annual CO2\nSavings (ton)"]
        ]

        # Populate table rows
        for _, row in summary_table.iterrows():
            matched_place_name = str(row["Matched Place Name"])
            if len(matched_place_name) > 20:
                matched_place_name = matched_place_name[:20] + "..."
            table_data.append([
                matched_place_name,
                str(row["count"]),
                row["maxArrayPanelsCount"],
                row["nominalPowerW"],
                row["predictedMeanAnnualPower"],
                row["housesPowered"],
                row["annualCO2Savings"]
            ])

        # Create and style the table
        table = Table(table_data, colWidths=[1.8 * inch, 0.6 * inch, 1 * inch, 1 * inch, 1 * inch, 0.8 * inch, 1.2 * inch])
        table.setStyle(TableStyle([
            ('BACKGROUND', (0, 0), (-1, 0), colors.grey),
            ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
            ('ALIGN', (1, 1), (-1, -1), 'CENTER'),
            ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
            ('FONTSIZE', (0, 0), (-1, 0), 8),
            ('FONTSIZE', (0, 1), (-1, -1), 7),
            ('BOTTOMPADDING', (0, 0), (-1, 0), 8),
            ('GRID', (0, 0), (-1, -1), 0.5, colors.black),
            ('ROWBACKGROUNDS', (0, 1), (-1, -1), [colors.whitesmoke, colors.lightgrey])
        ]))

        # Add elements to the document
        elements = [Paragraph(f"Community Solar Report for {town_name}, IN", title_style), Spacer(1, 12)]
        if map_image_path:
            elements.append(Image(map_image_path, width=6 * inch, height=4 * inch))
            elements.append(Spacer(1, 12))
        elements.append(table)
        elements.append(Spacer(1, 12))
        for chart in charts:
            elements.append(Image(chart, width=6 * inch, height=3 * inch))
            elements.append(Spacer(1, 12))

        # Build the PDF
        doc.build(elements)
        print(f"Report for {town_name} generated successfully at {pdf_file_path}!")

# Example usage
# generate_town_reports(df, ["Town1", "Town2"], "./output_reports", "./maps")

In [19]:
generate_town_reports(df, ["MONTEZUMA", "HUNTINGBURG", "LINTON", "WASHINGTON"], "./output_reports", "./maps")

Report for MONTEZUMA generated successfully at ./output_reports/MONTEZUMA/MONTEZUMA_report.pdf!
Report for HUNTINGBURG generated successfully at ./output_reports/HUNTINGBURG/HUNTINGBURG_report.pdf!
Report for LINTON generated successfully at ./output_reports/LINTON/LINTON_report.pdf!
Report for WASHINGTON generated successfully at ./output_reports/WASHINGTON/WASHINGTON_report.pdf!
